In [12]:
# Problem 3

# import libraries
import numpy as np
import torch, os, copy, shutil
import torch.nn as nn
from torch.utils.data import DataLoader , random_split
from torchvision import models, datasets, transforms
from torchvision.datasets import ImageFolder
import pandas as pd
from sklearn.metrics import confusion_matrix

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [13]:
# will change later lmao literally right from dhruv's rn lmao sorry
# i mean i use google colab so im 90% sure my issue is this right here lol

base_dir = "Cat Vs Dog Dataset"
train_dir = os.path.join(base_dir,"train")

if not os.path.exists(f"{train_dir}/cat"):
        os.makedirs(f"{train_dir}/cat")
        
if not os.path.exists(f"{train_dir}/dog"):
        os.makedirs(f"{train_dir}/dog")

# sort the cats and dogs into two separate files
onlyFilesinTrain = [filename for filename in os.listdir(train_dir) if not os.path.isdir(os.path.join(train_dir,filename))]

for filename in onlyFilesinTrain:
        
        label = filename.split(".")[0]
        
        if(label == "cat"):
                shutil.move(f"{train_dir}/{filename}" , f"{train_dir}/cat/{filename}")
                
        elif(label == "dog"):
                shutil.move(f"{train_dir}/{filename}" , f"{train_dir}/dog/{filename}")
        
        else:
                print("No Label")

In [14]:
# set parameters and loaders
batches = 50
epochs = 5          
classes = 2           # cat and dog

data_transform = transforms.Compose( [transforms.Resize((224, 224)), transforms.ToTensor()])
dataset = datasets.ImageFolder(train_dir,transform=data_transform)
size = len(dataset)

train_data,test_data = random_split(dataset, [int(0.7*size) , int(0.3*size)] ) # train 70%, test 30%


train_loader = DataLoader(dataset=train_data, batch_size=batches, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size = batches, shuffle=False)

dataset_classes = dataset.classes 

In [15]:
# load ResNet18 and evaluate the model
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
print(resnet18.fc)

resnet18.fc = nn.Linear(resnet18.fc.in_features,2) # only need 2 classes (cat and dog)
print()
print(resnet18.fc)

resnet18 = resnet18.to(device)

Linear(in_features=512, out_features=1000, bias=True)

Linear(in_features=512, out_features=2, bias=True)


In [16]:
# compare true and predicted label
def compute_accuracy (model, data_loader, device):

    model = model.to(device)
    model = model.eval()
    
    num_correct_prediction = 0
    num_total_labels = 0
    
    with torch.no_grad():
        
        for i, (inputs, labels) in enumerate(data_loader):
            inputs = inputs.to(device)
            labels = labels
            
            probabilities = resnet18(inputs.to(device))
            predicted_class = torch.argmax(probabilities.cpu(), dim=1)
            
            num_total_labels += labels.size()[0]
            num_correct_prediction += (predicted_class == labels).sum()

    return num_correct_prediction/num_total_labels * 100

In [17]:
# train ResNet18 model
def train_model (model, data_loader, learning_rate, num_epochs, device):
    
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    best_weights = copy.deepcopy(model.state_dict())
    best_accuracy = 0
    
    for epochs in range(num_epochs):
        
        print(f"\nEPOCH: {epochs+1}\n")
      
        model.train()
        for batch_index, (inputs, labels) in enumerate(data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)   
             
            probabilities = model(inputs)
            
            optimizer.zero_grad()
            loss = nn.functional.cross_entropy(probabilities,labels)
                
            loss.backward()
            optimizer.step()
            
            
            if (batch_index % 50 == 0) :
                print(f"Batch: {batch_index}/{len(data_loader)}    Loss: {loss}")
            
        model.eval()      # evaluate the model
        with torch.set_grad_enabled(False):
            accuracy = compute_accuracy(model,data_loader,device)
            print(f"\nAccuracy: {accuracy}%\n")
            print("-----------------------------------")
        if (accuracy > best_accuracy):
            best_accuracy = accuracy
            best_weights = model.state_dict()
    
    model.load_state_dict(best_weights)
    return model

resnet18 = train_model(resnet18,train_loader,0.001,epochs,device)


EPOCH: 1

Batch: 0/350    Loss: 0.8426664471626282
Batch: 50/350    Loss: 0.3809688687324524
Batch: 100/350    Loss: 0.30176690220832825
Batch: 150/350    Loss: 0.23191529512405396
Batch: 200/350    Loss: 0.14254559576511383
Batch: 250/350    Loss: 0.17372894287109375
Batch: 300/350    Loss: 0.1602909117937088

Accuracy: 97.36571502685547%

-----------------------------------

EPOCH: 2

Batch: 0/350    Loss: 0.0874902606010437
Batch: 50/350    Loss: 0.08381566405296326
Batch: 100/350    Loss: 0.1120864599943161
Batch: 150/350    Loss: 0.10752487927675247
Batch: 200/350    Loss: 0.1666588932275772
Batch: 250/350    Loss: 0.05690792575478554
Batch: 300/350    Loss: 0.05321763828396797

Accuracy: 98.07428741455078%

-----------------------------------

EPOCH: 3

Batch: 0/350    Loss: 0.13832172751426697
Batch: 50/350    Loss: 0.03512366861104965
Batch: 100/350    Loss: 0.12975898385047913
Batch: 150/350    Loss: 0.10037980228662491
Batch: 200/350    Loss: 0.054456695914268494
Batch: 250/

In [18]:
# test ResNet18 after training
predicted_class18 = torch.empty(0).to(device)
true_label_18 = torch.empty(0)

resnet18 = resnet18.eval()

for batch_index, (inputs, labels) in enumerate(test_loader):

    true_label_18 = torch.cat((true_label_18,labels))
    
    with torch.no_grad():
        probabilities = resnet18(inputs.to(device))
        predicted_class18 = torch.cat((predicted_class18,torch.argmax(probabilities,dim=1)))

with torch.no_grad():
        prob = resnet18(inputs.to(device))
        pred_class = torch.argmax(prob, dim=1)

print(f"\n\nPredicted Class : {dataset_classes[int(pred_class[10].item())]}")
print(f"Actual Class : {dataset_classes[int(labels[10].item())]}")
print(f"\nTotal Accuracy : {(compute_accuracy(resnet18,test_loader,device))}%\n")



Predicted Class : dog
Actual Class : dog

Total Accuracy : 98.36000061035156%



In [21]:
# sklearn results - resnet18
y_true = true_label_18.to('cpu')
y_pred = predicted_class18.to('cpu')

class_label = ["cat","dog"]

confusion = confusion_matrix(y_true,y_pred)

cfm = pd.DataFrame(confusion,index=class_label, columns=class_label)
print("\n\nConfusion Matrix for all 10,000 samples:\n")
print(cfm)

tp = np.array([confusion[i][i] for i in range(len(confusion[0]))])
fp = np.array([sum(confusion[:,i]) for i in range(len(confusion[0]))]) - tp
fn = np.array([sum(confusion[i,:]) for i in range(len(confusion[0]))])  - tp
tn = np.array([sum(sum(confusion)) for i in range(len(confusion[0]))]) - tp - fp - fn

precision = tp / (fp+tp)
recall = tp / (fn+tp)
fscore = 2 * (precision * recall)/(precision +recall)
accuracy = tp / (fn+tp)

# showing average values
print(f"\nAccuracy: {sum(accuracy)/len(accuracy)}") 
print(f"F-score: {sum(fscore)/len(fscore)}")
print(f"Precison: {sum(precision)/len(precision)}")
print(f"Recall: {sum(recall)/len(recall)}")



Confusion Matrix for all 10,000 samples:

      cat   dog
cat  3612    47
dog    76  3765

Accuracy: 0.983684223221511
F-score: 0.9835931721345155
Precison: 0.9835315694473212
Recall: 0.983684223221511
